**Key COnsideration for Dataproc Cluster

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Olist Ecommernce Optimization').getOrCreate()


25/12/26 01:09:47 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Olist Ecommernce Optimization')\
.config('spark.executor.memory','2g') \
.config('spark.executor.cores','2') \
.config('spark.executor.instances', '2') \
.config('spark.driver.memory', '2g')\
.config('spark.driver.maxResultsSize','2g') \
.config('spark.sql.shuffle.partitions', '64')\
.config('spark.default.parallelism', '64') \
.config('spark.sql.adaptive.enabled', 'true') \
.config('spark.sql.adaptive.coalescePartition.enabled', 'true') \
.config('spark.sql.autoBroadcastJoinThreshold',20*1024*1024) \
.config('spark.sql.files.maxPartitionBytes', '64MB') \
.config('spark.sql.files.openCOstInBytes', '2MB') \
.config('spark.memory.fraction', 0.8) \
.config('spark.memory.storageFraction', 0.2) \
.getOrCreate()






25/12/26 01:11:34 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *

In [6]:
path = '/data/'
geolocation = spark.read.csv(path+'olist_geolocation_dataset.csv',header=True,inferSchema = True)
order_items = spark.read.csv(path+'olist_order_items_dataset.csv',header=True,inferSchema = True)
payments = spark.read.csv(path+'olist_order_payments_dataset.csv',header=True,inferSchema = True)
reviews = spark.read.csv(path+'olist_order_reviews_dataset.csv',header=True,inferSchema = True)
orders = spark.read.csv(path+'olist_orders_dataset.csv',header=True,inferSchema = True)
products = spark.read.csv(path+'olist_products_dataset.csv',header=True,inferSchema = True)
seller = spark.read.csv(path+'olist_sellers_dataset.csv',header=True,inferSchema = True)
customer = spark.read.csv(path+'olist_customers_dataset.csv',header=True,inferSchema = True)

**Optimize Join Strategies

In [8]:
#Broadcast



broadcast_customer = broadcast(customer)
optimized_broadcast_join = all_dataframe.join(broadcast_customer,'customer_id')



In [9]:
#Sort And Merge Join

sorted_customer = customer.sortWithinPartitions('customer_id')
sorted_orders = all_dataframe.sortWithinPartitions('customer_id')


optimized_merge_all_dataframe = sorted_orders.join(sorted_customer,'customer_id')

In [ ]:
#Bucket Join

bucketed_customer = customer.repartition(10, 'customer_id')
bucketed_order = all_dataframe.repartition(10, 'customer_id')

bucketed_all_dataframe = bucketed_order.joing(bucketed_customer,'customer_id')

In [10]:
olist_full = spark.read.parquet('/data/processed')

In [11]:
olist_full.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- payment_sequential: integer (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- payment_installments: integer (nullable = true)
 |-- payment_value: double (nullable = true)
 |-- review_id: string (nullable = true)
 |-- review_score: string (nullable = true)
 |-- review_comment_tit

In [ ]:
#Caching